In [702]:
import re
import math
import itertools
import random
file = open('input16', 'r')
lines = file.read().splitlines()

class Node():
    def __init__(self, name, children, flow):
        self.name = name
        self.children = children
        self.flow = flow
        self.index = 0
        
    def __eq__(self, other):
        return self.name == other.name   
    
    def __lt__(self, other):
        return self.flow < other.flow
    
    def __str__(self):
        return self.name + "(" + str(self.flow) + ")"

In [703]:
nodes = []

def nodeByName(name):
    return [node for node in nodes if node.name == name][0]

def pathLength(source, destination):
    for node in nodes: 
        node.depth = None 
    source.depth = 0
    queue = [source]
    while len(queue) > 0:        
        node = queue.pop(0)
        if node == destination:
            return node.depth
        for child in node.children:
            if child.depth == None:
                child.depth = node.depth + 1
                queue += [child]      
                
def pathLengthByName(source, destination):  
    return pathLength(nodeByName(source), nodeByName(destination))

for i, line in enumerate(lines):
    flow = int(re.findall("\d+", line)[0])    
    name = re.findall("[A-Z]+", line)[1]
    node = Node(name, [], flow)
    node.identifier = pow(2, i)
    nodes.append(node)
    
for i, line in enumerate(lines):
    childrenNames = re.findall("[A-Z]+", line)[2:]
    for childName in childrenNames:        
        childNode = [node for node in nodes if node.name == childName][0]
        nodes[i].children.append(childNode)    
        
root = nodeByName('AA')

In [704]:
flowNodes = list(filter(lambda node: node.flow > 0, nodes))
    
distances = []
for index, node in enumerate([root] + flowNodes):    
    distances.append([pathLength(node, other) for other in [root] + flowNodes])
    node.index = index

In [727]:
def findMaxFlow(nodes, time):
    flows = [0]
    histories = [[]]    
    queue = [(root, 0, nodes.copy(), 0, [root])]
    while len(queue) > 0:
        (node, elapsed, remainingNodes, combinedFlow, history) = queue.pop(0)
        remainingNodes.remove(node)
        
        if elapsed >= 20 and combinedFlow < 1100:
            continue                 

        if len(remainingNodes) == 0:
            combinedFlow += (time-elapsed) * node.flow            
            
        if elapsed >= time or len(remainingNodes) == 0:
            flows.append(combinedFlow)
            histories.append(history) 
        else:
            combinedFlow += (time-elapsed) * node.flow            
            for nextNode in remainingNodes:          
                nextElapsed = elapsed + distances[nextNode.index][node.index] + 1
                nextRemainingNodes = remainingNodes.copy()
                queue.append((nextNode, nextElapsed, nextRemainingNodes, combinedFlow, history + [nextNode]))
        
    maxFlow = max(flows)  
    return (maxFlow, histories[flows.index(maxFlow)])

In [725]:
findMaxFlow([root] + flowNodes, 30)[0]

2080

In [733]:
results = []
histories = []
for combination in itertools.combinations(list(range(15)), 7):
    a = []
    b = []
    for index in combination:
        a.append(flowNodes[index])
    for node in flowNodes:
        if not node in a:
            b.append(node)
            
    (resultA, historyA) = findMaxFlow([root] + a, 26)
    (resultB, historyB) = findMaxFlow([root] + b, 26)    
    results.append(resultA+resultB)
    histories.append((historyA, historyB))
max(results)

2708